In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
tree_df = pd.DataFrame(data = {"Daughter_1": [2, 1, 4, 11, 10, 7, 13],
                               "Branch_Length_1": [2.7, 1.8, 1.1, 1.9, 1.5, 4.7, 2.2],
                               "Daughter_2": [3, 9, 5, 6, 12, 8, 14],
                               "Branch_Length_2": [2.7, 4.5, 1.1, 3, 3, 4.7, 3.5]},
                       index = [9, 10, 11, 12, 13, 14, 15])

display(tree_df)

,Daughter_1,Branch_Length_1,Daughter_2,Branch_Length_2
9,2,2.7,3,2.7
10,1,1.8,9,4.5
11,4,1.1,5,1.1
12,11,1.9,6,3.0
13,10,1.5,12,3.0
14,7,4.7,8,4.7
15,13,2.2,14,3.5


In [3]:
class TreeInference():
    """
    tree_data: dataframe of nodes, daughters and branch length
    mu: mutation rate
    leaf_values: bases for leaf nodes
    """
    def __init__(self, tree_data, mu, leaf_values):
        # Set parameters
        self.tree_data = tree_data.copy()
        self.mu = mu
        self.leaf_values = leaf_values
        
        # Find IDs of internal nodes, all nodes and leaf nodes
        self.internal_node_ids = np.array(tree_data.index)
        self.all_node_ids = np.unique(np.concatenate((np.array(self.tree_data.index),
                                                      np.array(self.tree_data["Daughter_1"]),
                                                      np.array(self.tree_data["Daughter_2"]))))
        self.leaf_node_ids = list(set(self.all_node_ids) - set(self.internal_node_ids))
        
        # Record branch lengths to leaves
        self.root_leaf_dist = {}
        
        # Count internal nodes and leaves for each row in tree_data
        self.treeNodeCounts()
        
        # Get indexes for Q and A matricies for states, e.g. A is row 0, C is row 2...
        states = np.unique(list(leaf_values.values()))
        self.matrix_idx = dict(zip(states, range(len(states))))
        
        # Create Q matrix for bases A, C, T, G
        self.Q = self.MatrixQ()
        # Create empty matrix of likelihood for each node
        self.L = np.full((len(self.all_node_ids), len(self.matrix_idx)), None)
        
    # For each internal node, calculate the number of internal nodes and leaves below
    def treeNodeCounts(self):
        n_nodes = np.array([])
        n_leaves = np.array([])
        max_leaves = 0

        # Set the number of internal nodes and leaves for each internal node row
        for n in self.internal_node_ids:
            num_nodes, num_leaves, leaf_dist = self.checkNode(self.tree_data,
                                                              n,
                                                              self.internal_node_ids)
            n_nodes = np.append(n_nodes, num_nodes)
            n_leaves = np.append(n_leaves, num_leaves)

            # Check if found root node
            if len(leaf_dist) > max_leaves:
                # Record branch lengths to leaves
                self.root_leaf_dist = leaf_dist
                max_leaves = len(leaf_dist)

        # Add new information to dataframe
        self.tree_data["N_Internal"] = n_nodes
        self.tree_data["N_Leaves"] = n_leaves
                
    # Recursively find the number of internal nodes and leaves given a node ID
    def checkNode(self, tree_data, node_id, node_array):
        # Record the number of internal nodes and leaves below
        num_nodes = 0
        num_leaves = 0
        # Record branch lengths of leaves below
        leaf_lengths = {}

        # Find the left and right daughter nodes
        daughter_1 = self.tree_data.loc[node_id]["Daughter_1"]
        daughter_2 = self.tree_data.loc[node_id]["Daughter_2"]

        # Check if found an internal node to the left
        if daughter_1 in node_array:
            # Recursively check nodes below
            d1_nodes, d1_leaves, d1_leaf_lengths = self.checkNode(self.tree_data,
                                                                  daughter_1, node_array)
            num_nodes += d1_nodes + 1
            num_leaves += d1_leaves

            # Increment branch lengths of leaves
            for l in d1_leaf_lengths.keys():
                leaf_lengths[l] = round(d1_leaf_lengths[l] + tree_data.loc[node_id]["Branch_Length_1"], 1)

        # Otherwise found a left leaf node
        else:
            num_leaves += 1
            leaf_lengths[daughter_1] = self.tree_data.loc[node_id]["Branch_Length_1"]

        # Check if found an internal node to the right
        if daughter_2 in node_array:
            d2_nodes, d2_leaves, d2_leaf_lengths = self.checkNode(self.tree_data,
                                                                  daughter_2,
                                                                  node_array)
            num_nodes += d2_nodes + 1
            num_leaves += d2_leaves

            for l in d2_leaf_lengths.keys():
                leaf_lengths[l] = round(d2_leaf_lengths[l] + self.tree_data.loc[node_id]["Branch_Length_2"], 1)

        # Otherwise found a right leaf node
        else:
            num_leaves += 1
            leaf_lengths[daughter_2] = self.tree_data.loc[node_id]["Branch_Length_2"]

        return num_nodes, num_leaves, leaf_lengths
    
    # Jukes-Cantor model Q matrix
    def MatrixQ(self):
        # Create matrix with all off-diagonal rates in Q as the same value
        Q = np.full((len(self.matrix_idx), len(self.matrix_idx)),
                    self.mu/(len(self.matrix_idx) - 1))
        np.fill_diagonal(Q, 1 - self.mu)
        return Q
    
    # Create transition matrix for time t
    def TransitionMatrixA(self, t, mu):
        A = np.full((4, 4), (1/4) * (1 - math.exp(-4 * mu * t)))
        np.fill_diagonal(A, (1/4) * (1 + 3 * math.exp(-4 * mu * t)))
        return A
    
    def nodeLikelihood(self, node_n):
        # Find the left and right daughter nodes
        daughter_1 = self.tree_data.loc[node_n]["Daughter_1"]
        daughter_2 = self.tree_data.loc[node_n]["Daughter_2"]
        
        # Check if likelihood of left child node set
        if (self.L[int(daughter_1) - 1, 0] is None):
            # If not set then calculate child likelihood first
            self.nodeLikelihood(int(daughter_1))
        # Check if likelihood of right child node set
        if (self.L[int(daughter_2) - 1, 0] is None):
            self.nodeLikelihood(int(daughter_2))
        
        # Get branch length times for child nodes
        t1 = self.tree_data.loc[node_n]["Branch_Length_1"]
        t2 = self.tree_data.loc[node_n]["Branch_Length_2"]
        
        # Create transition matrix for left and right child for t1 and t2
        A1 = self.TransitionMatrixA(t1, self.mu)
        A2 = self.TransitionMatrixA(t2, self.mu)
        
        # Get conditional likelihood of child nodes
        d1_likelihood = self.L[int(daughter_1) - 1]
        d2_likelihood = self.L[int(daughter_2) - 1]
        
        # Calculate the likelihood of each state for the node
        left_prob = np.sum(A1 * d1_likelihood, axis = 1)
        right_prob = np.sum(A2 * d2_likelihood, axis = 1)
        self.L[node_n - 1] = np.multiply(left_prob, right_prob)
    
    def calculateLikelihood(self):
        # Set likelihood of leaf states
        for leaf_n in self.leaf_node_ids:
            # Zero for non-matching base
            self.L[leaf_n - 1] = 0
            # One for matching base
            self.L[leaf_n - 1, self.matrix_idx[self.leaf_values[leaf_n]]] = 1
        
        for node_n in self.internal_node_ids:
            # Check likelihood has not been set
            if self.L[node_n - 1, 0] is None:
                # Calculate conditional likelihood of each state for the node
                self.nodeLikelihood(node_n)

In [4]:
tree_df2 = tree_df.append(pd.DataFrame({"Daughter_1": [16, 18],
                          "Branch_Length_1": [2.7, 1.8],
                          "Daughter_2": [17, 19],
                          "Branch_Length_2": [4, 1.4]},
                        index = [1, 3]))

tree_leaves2 = {1: "A", 2: "G", 3: "T", 4: "G", 5: "G", 6: "G", 7: "C", 8: "G",
                16: "C", 17: "A", 18: "A", 19: "G"}

display(tree_df2)

,Daughter_1,Branch_Length_1,Daughter_2,Branch_Length_2
9,2,2.7,3,2.7
10,1,1.8,9,4.5
11,4,1.1,5,1.1
12,11,1.9,6,3.0
13,10,1.5,12,3.0
14,7,4.7,8,4.7
15,13,2.2,14,3.5
1,16,2.7,17,4.0
3,18,1.8,19,1.4


In [5]:
tree_leaves = {1: "A", 2: "G", 3: "T", 4: "G", 5: "G", 6: "G", 7: "C", 8: "G"}

inference = TreeInference(tree_df2, mu = 0.01,
                          leaf_values = tree_leaves2)

display(inference.tree_data)

# print("Distance for each leaf from root:")
# print(inference.root_leaf_dist)

# print("\nQ matrix: \n" + str(inference.Q))

# print("\nP matrix: \n" + str(inference.P))

print(inference.matrix_idx)

inference.calculateLikelihood()


inference.L


,Daughter_1,Branch_Length_1,Daughter_2,Branch_Length_2,N_Internal,N_Leaves
9,2,2.7,3,2.7,1.0,3.0
10,1,1.8,9,4.5,3.0,5.0
11,4,1.1,5,1.1,0.0,2.0
12,11,1.9,6,3.0,1.0,3.0
13,10,1.5,12,3.0,6.0,8.0
14,7,4.7,8,4.7,0.0,2.0
15,13,2.2,14,3.5,8.0,10.0
1,16,2.7,17,4.0,0.0,2.0
3,18,1.8,19,1.4,0.0,2.0


{'A': 0, 'C': 1, 'G': 2, 'T': 3}


array([[0.022755026697475443, 0.03412597856353133, 0.0009460247316386124,
        0.0009460247316386124],
       [0, 0, 1, 0],
       [0.012905838381830586, 0.00023645921519011413,
        0.01665789840162822, 0.00023645921519011413],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0.00031616098933535925, 2.5106408776689784e-05,
        0.014514248429786634, 2.5106408776689784e-05],
       [1.9462787356928317e-05, 2.0773139585450094e-05,
        2.42118233962085e-05, 1.2048578145710967e-06],
       [0.00011581011107701711, 0.00011581011107701711,
        0.9364732272092632, 0.00011581011107701711],
       [0.0004875809010589545, 0.0004875809010589545, 0.8100153613472278,
        0.0004875809010589546],
       [4.5074972545723367e-07, 4.795928245594715e-07,
        1.761297179450265e-05, 4.886139365658112e-08],
       [0.0018358074110587364, 0.037338900958653644,
        0.037338900958653644, 0.0018358074110587364],
      